In [38]:
import pandas as pd
import numpy as np

import scipy.optimize as opt

from pathlib import Path
root = Path("../input/google-smartphone-decimeter-challenge/")

In [39]:
def ecef2lla(x, y, z):
    # x, y and z are scalars or vectors in meters
    x = np.array([x]).reshape(np.array([x]).shape[-1], 1)
    y = np.array([y]).reshape(np.array([y]).shape[-1], 1)
    z = np.array([z]).reshape(np.array([z]).shape[-1], 1)

    a=6378137
    a_sq=a**2
    e = 8.181919084261345e-2
    e_sq = 6.69437999014e-3

    f = 1/298.257223563
    b = a*(1-f)

    # calculations:
    r = np.sqrt(x**2 + y**2)
    ep_sq  = (a**2-b**2)/b**2
    ee = (a**2-b**2)
    f = (54*b**2)*(z**2)
    g = r**2 + (1 - e_sq)*(z**2) - e_sq*ee*2
    c = (e_sq**2)*f*r**2/(g**3)
    s = (1 + c + np.sqrt(c**2 + 2*c))**(1/3.)
    p = f/(3.*(g**2)*(s + (1./s) + 1)**2)
    q = np.sqrt(1 + 2*p*e_sq**2)
    r_0 = -(p*e_sq*r)/(1+q) + np.sqrt(0.5*(a**2)*(1+(1./q)) - p*(z**2)*(1-e_sq)/(q*(1+q)) - 0.5*p*(r**2))
    u = np.sqrt((r - e_sq*r_0)**2 + z**2)
    v = np.sqrt((r - e_sq*r_0)**2 + (1 - e_sq)*z**2)
    z_0 = (b**2)*z/(a*v)
    h = u*(1 - b**2/(a*v))
    phi = np.arctan((z + ep_sq*z_0)/r)
    lambd = np.arctan2(y, x)

    return phi*180/np.pi, lambd*180/np.pi, h

def calc_haversine(lat1, lon1, lat2, lon2):
    """Calculates the great circle distance between two points
    on the earth. Inputs are array-like and specified in decimal degrees.
    """
    RADIUS = 6_367_000
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + \
      np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    dist = 2 * RADIUS * np.arcsin(a**0.5)
    return dist

# Apply WLS on one collection and one measurement

In [40]:
collection_name="2020-05-29-US-MTV-1"
file_path = Path(f"train/{collection_name}")
phone = 'Pixel4'
measurement_epoch_time = 1274827487438

# baseline we'll compare our solution against
df_baseline = pd.read_csv(root/"baseline_locations_train.csv")

# ground truth to compute methods performance
df_groundtruth = pd.read_csv(root/file_path/f"{phone}/ground_truth.csv")

# Train df here only contains one collection and one measurement
df_train = pd.read_csv(root/file_path/f"{phone}/{phone}_derived.csv")
df_train = df_train[df_train['millisSinceGpsEpoch'] == measurement_epoch_time] 

In [14]:
# Corrected pseudorange according to data instructions
df_train['correctedPrM'] = df_train.apply(
    lambda r: r.rawPrM + r.satClkBiasM - r.isrbM - r.ionoDelayM - r.tropoDelayM,
    axis=1
)

# Time it took for signal to travel
light_speed = 299_792_458
df_train['transmissionTimeSeconds'] = df_train['correctedPrM'] / light_speed

In [15]:
# Compute true sat positions at arrival time
omega_e = 7.2921151467e-5
df_train['xSatPosMRotated'] = \
    np.cos(omega_e * df_train['transmissionTimeSeconds']) * df_train['xSatPosM'] \
    + np.sin(omega_e * df_train['transmissionTimeSeconds']) * df_train['ySatPosM']
    
df_train['ySatPosMRotated'] = \
    - np.sin(omega_e * df_train['transmissionTimeSeconds']) * df_train['xSatPosM'] \
    + np.cos(omega_e * df_train['transmissionTimeSeconds']) * df_train['ySatPosM']
    
df_train['zSatPosMRotated'] = df_train['zSatPosM']

In [16]:
# Uncertainty weight for the WLS method
df_train['uncertaintyWeight'] = 1 / df_train['rawPrUncM']

In [17]:
# Set up least squares methods
def distance(sat_pos, x):
    sat_pos_diff = sat_pos.copy(deep=True)
    
    sat_pos_diff['xSatPosMRotated'] = sat_pos_diff['xSatPosMRotated'] - x[0]
    sat_pos_diff['ySatPosMRotated'] = sat_pos_diff['ySatPosMRotated'] - x[1]
    sat_pos_diff['zSatPosMRotated'] = sat_pos_diff['zSatPosMRotated'] - x[2]

    sat_pos_diff['d'] = sat_pos_diff.apply(
        lambda r: r.uncertaintyWeight * 
            (np.sqrt((r.xSatPosMRotated**2 + r.ySatPosMRotated**2 + r.zSatPosMRotated**2)) + x[3] - r.correctedPrM),
        axis=1
    )

    return sat_pos_diff['d']

def distance_fixed_satpos(x):
    return distance(df_train[['xSatPosMRotated', 'ySatPosMRotated', 'zSatPosMRotated', 'correctedPrM', 'uncertaintyWeight']], x)

In [18]:
# Start point for the optimiser
x0= [0,0,0,0]

opt_res = opt.least_squares(distance_fixed_satpos, x0)

# Optimiser yields a position in the ECEF coordinates
opt_res_pos = opt_res.x

In [19]:
# ECEF position to lat/long
wls_estimated_pos = ecef2lla(*opt_res_pos[:3])
wls_estimated_pos = np.squeeze(wls_estimated_pos)

In [21]:
val_baseline = df_baseline[
    (df_baseline['collectionName']==collection_name)
    & (df_baseline['phoneName']==phone)
    & (df_baseline['millisSinceGpsEpoch']==measurement_epoch_time)
].iloc[0]

In [22]:
val_groundtruth = df_groundtruth[
    (df_groundtruth['collectionName']==collection_name)
    & (df_groundtruth['phoneName']==phone)
    & (df_groundtruth['millisSinceGpsEpoch']==measurement_epoch_time)
].iloc[0]

In [24]:
print("Baseline distance with groundtruth position (m)")
calc_haversine(val_baseline['latDeg'], val_baseline['lngDeg'], val_groundtruth['latDeg'], val_groundtruth['lngDeg'])

Baseline distance with groundtruth position (m)


15.419474998287948

In [25]:
print("Our estimated position (with WLS) distance with groundtruth position (m)")
calc_haversine(wls_estimated_pos[0], wls_estimated_pos[1], val_groundtruth['latDeg'], val_groundtruth['lngDeg'])

Our estimated position (with WLS) distance with groundtruth position (m)


11.161633363336618

In [26]:
val_baseline

collectionName                       2020-05-29-US-MTV-1
phoneName                                         Pixel4
millisSinceGpsEpoch                        1274827487438
latDeg                                         37.416644
lngDeg                                       -122.080149
heightAboveWgs84EllipsoidM                        -21.74
phone                         2020-05-29-US-MTV-1_Pixel4
Name: 11624, dtype: object

In [27]:
wls_estimated_pos

array([  37.4166094 , -122.08003187,  -29.53142955])

,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,heightAboveWgs84EllipsoidM,phone
0,2020-05-14-US-MTV-1,Pixel4,1273529463442,37.423575,-122.094091,-34.06,2020-05-14-US-MTV-1_Pixel4
1,2020-05-14-US-MTV-1,Pixel4,1273529464442,37.423578,-122.094101,-33.29,2020-05-14-US-MTV-1_Pixel4
2,2020-05-14-US-MTV-1,Pixel4,1273529465442,37.423573,-122.094111,-30.99,2020-05-14-US-MTV-1_Pixel4
3,2020-05-14-US-MTV-1,Pixel4,1273529466442,37.423583,-122.094121,-32.83,2020-05-14-US-MTV-1_Pixel4
4,2020-05-14-US-MTV-1,Pixel4,1273529467442,37.423579,-122.094114,-34.49,2020-05-14-US-MTV-1_Pixel4
...,...,...,...,...,...,...,...
131337,2021-04-29-US-SJC-2,SamsungS20Ultra,1303760315000,37.334460,-121.899600,-8.09,2021-04-29-US-SJC-2_SamsungS20Ultra
131338,2021-04-29-US-SJC-2,SamsungS20Ultra,1303760316000,37.334472,-121.899583,-7.59,2021-04-29-US-SJC-2_SamsungS20Ultra
131339,2021-04-29-US-SJC-2,SamsungS20Ultra,1303760317000,37.334491,-121.899597,-8.35,2021-04-29-US-SJC-2_SamsungS20Ultra
131340,2021-04-29-US-SJC-2,SamsungS20Ultra,1303760318000,37.334495,-121.899583,-8.73,2021-04-29-US-SJC-2_SamsungS20Ultra


In [29]:
base_test = pd.read_csv(root/"baseline_locations_test.csv")

In [32]:
base_test['millisSinceGpsEpoch']==1273608786432

0        False
1         True
2        False
3        False
4        False
         ...  
91481    False
91482    False
91483    False
91484    False
91485    False
Name: millisSinceGpsEpoch, Length: 91486, dtype: bool

In [35]:
base_test

,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,heightAboveWgs84EllipsoidM,phone
0,2020-05-15-US-MTV-1,Pixel4,1273608785432,37.416628,-122.082053,-30.69,2020-05-15-US-MTV-1_Pixel4
1,2020-05-15-US-MTV-1,Pixel4,1273608786432,37.416646,-122.082040,-31.76,2020-05-15-US-MTV-1_Pixel4
2,2020-05-15-US-MTV-1,Pixel4,1273608787432,37.416652,-122.082039,-31.65,2020-05-15-US-MTV-1_Pixel4
3,2020-05-15-US-MTV-1,Pixel4,1273608788432,37.416607,-122.082063,-31.52,2020-05-15-US-MTV-1_Pixel4
4,2020-05-15-US-MTV-1,Pixel4,1273608789432,37.416609,-122.082073,-28.95,2020-05-15-US-MTV-1_Pixel4
...,...,...,...,...,...,...,...
91481,2021-04-29-US-SJC-3,SamsungS20Ultra,1303763185000,37.334539,-121.899383,-8.39,2021-04-29-US-SJC-3_SamsungS20Ultra
91482,2021-04-29-US-SJC-3,SamsungS20Ultra,1303763186000,37.334545,-121.899380,-7.36,2021-04-29-US-SJC-3_SamsungS20Ultra
91483,2021-04-29-US-SJC-3,SamsungS20Ultra,1303763187000,37.334551,-121.899371,-4.08,2021-04-29-US-SJC-3_SamsungS20Ultra
91484,2021-04-29-US-SJC-3,SamsungS20Ultra,1303763188000,37.334540,-121.899371,-5.70,2021-04-29-US-SJC-3_SamsungS20Ultra


In [33]:
sub = pd.read_csv(root/"sample_submission.csv")

In [34]:
sub

,phone,millisSinceGpsEpoch,latDeg,lngDeg
0,2020-05-15-US-MTV-1_Pixel4,1273608785432,37.904611,-86.481078
1,2020-05-15-US-MTV-1_Pixel4,1273608786432,37.904611,-86.481078
2,2020-05-15-US-MTV-1_Pixel4,1273608787432,37.904611,-86.481078
3,2020-05-15-US-MTV-1_Pixel4,1273608788432,37.904611,-86.481078
4,2020-05-15-US-MTV-1_Pixel4,1273608789432,37.904611,-86.481078
...,...,...,...,...
91481,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763185000,37.904611,-86.481078
91482,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763186000,37.904611,-86.481078
91483,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763187000,37.904611,-86.481078
91484,2021-04-29-US-SJC-3_SamsungS20Ultra,1303763188000,37.904611,-86.481078


In [63]:
tmp = pd.read_csv(root/'test/2020-05-15-US-MTV-1/Pixel4/Pixel4_derived.csv')
tmp

,collectionName,phoneName,millisSinceGpsEpoch,constellationType,svid,signalType,receivedSvTimeInGpsNanos,xSatPosM,ySatPosM,zSatPosM,xSatVelMps,ySatVelMps,zSatVelMps,satClkBiasM,satClkDriftMps,rawPrM,rawPrUncM,isrbM,ionoDelayM,tropoDelayM
0,2020-05-15-US-MTV-1,Pixel4,1273608786431,1,2,GPS_L1,1273608785359142537,-1.355808e+07,-2.285482e+07,1.544018e+06,299.739,-52.442,3161.066,-136834.371,-0.002,2.176422e+07,4.197,0.000,4.792,3.446
1,2020-05-15-US-MTV-1,Pixel4,1273608786431,6,11,GAL_E1,1273608785345898271,1.275171e+07,-1.023081e+07,2.466967e+07,1759.975,1745.212,-184.065,855531.658,0.068,2.573475e+07,10.493,-214.130,6.801,6.527
2,2020-05-15-US-MTV-1,Pixel4,1273608786431,1,6,GPS_L1,1273608785362136210,-5.156902e+06,-2.381480e+07,1.056469e+07,820.205,1068.285,2827.140,-81575.023,-0.002,2.086674e+07,3.897,0.000,3.945,2.845
3,2020-05-15-US-MTV-1,Pixel4,1273608786431,1,12,GPS_L1,1273608785356867050,-1.745219e+07,1.887561e+06,1.964540e+07,1365.684,-2152.439,1418.596,35245.362,-0.002,2.244640e+07,6.296,0.000,5.739,4.806
4,2020-05-15-US-MTV-1,Pixel4,1273608786431,6,2,GAL_E5A,1273608785354070569,-8.979749e+06,-1.914611e+07,2.071128e+07,494.969,-2020.888,-1652.319,40044.541,0.001,2.328476e+07,1.799,-2344.573,6.341,2.547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57725,2020-05-15-US-MTV-1,Pixel4,1273612275632,3,13,GLO_G1,1273612275568180115,-1.219834e+07,-1.338065e+07,1.794654e+07,-705.894,-2360.645,-2237.702,-11610.677,0.000,1.927615e+07,4.197,1153.070,3.620,2.577
57726,2020-05-15-US-MTV-1,Pixel4,1273612275632,3,14,GLO_G1,1273612275557382661,-9.952700e+06,5.477318e+06,2.284841e+07,-718.551,-3041.501,414.853,15362.334,-0.000,2.251315e+07,4.797,1153.070,6.953,6.868
57727,2020-05-15-US-MTV-1,Pixel4,1273612275632,6,30,GAL_E1,1273612275555754963,-1.589697e+07,-7.064249e+06,2.393849e+07,22.881,-2404.899,-693.358,1170586.979,-0.009,2.300112e+07,5.096,-222.012,4.288,3.066
57728,2020-05-15-US-MTV-1,Pixel4,1273612275632,6,36,GAL_E1,1273612275552308959,6.525533e+05,-1.761160e+07,2.378304e+07,2044.321,1235.423,859.698,167805.129,-0.002,2.403420e+07,2.099,-222.012,4.311,3.081


In [67]:
base_test

,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,heightAboveWgs84EllipsoidM,phone
0,2020-05-15-US-MTV-1,Pixel4,1273608785432,37.416628,-122.082053,-30.69,2020-05-15-US-MTV-1_Pixel4
1,2020-05-15-US-MTV-1,Pixel4,1273608786432,37.416646,-122.082040,-31.76,2020-05-15-US-MTV-1_Pixel4
2,2020-05-15-US-MTV-1,Pixel4,1273608787432,37.416652,-122.082039,-31.65,2020-05-15-US-MTV-1_Pixel4
3,2020-05-15-US-MTV-1,Pixel4,1273608788432,37.416607,-122.082063,-31.52,2020-05-15-US-MTV-1_Pixel4
4,2020-05-15-US-MTV-1,Pixel4,1273608789432,37.416609,-122.082073,-28.95,2020-05-15-US-MTV-1_Pixel4
...,...,...,...,...,...,...,...
91481,2021-04-29-US-SJC-3,SamsungS20Ultra,1303763185000,37.334539,-121.899383,-8.39,2021-04-29-US-SJC-3_SamsungS20Ultra
91482,2021-04-29-US-SJC-3,SamsungS20Ultra,1303763186000,37.334545,-121.899380,-7.36,2021-04-29-US-SJC-3_SamsungS20Ultra
91483,2021-04-29-US-SJC-3,SamsungS20Ultra,1303763187000,37.334551,-121.899371,-4.08,2021-04-29-US-SJC-3_SamsungS20Ultra
91484,2021-04-29-US-SJC-3,SamsungS20Ultra,1303763188000,37.334540,-121.899371,-5.70,2021-04-29-US-SJC-3_SamsungS20Ultra


In [66]:
base_test[base_test['millisSinceGpsEpoch'].isin(tmp['millisSinceGpsEpoch'])]

,collectionName,phoneName,millisSinceGpsEpoch,latDeg,lngDeg,heightAboveWgs84EllipsoidM,phone
16,2020-05-15-US-MTV-1,Pixel4,1273608801654,37.416566,-122.082111,-27.75,2020-05-15-US-MTV-1_Pixel4
19,2020-05-15-US-MTV-1,Pixel4,1273608804648,37.416576,-122.082096,-35.79,2020-05-15-US-MTV-1_Pixel4
22,2020-05-15-US-MTV-1,Pixel4,1273608807665,37.416584,-122.082122,-27.05,2020-05-15-US-MTV-1_Pixel4
24,2020-05-15-US-MTV-1,Pixel4,1273608809647,37.416566,-122.082042,-24.89,2020-05-15-US-MTV-1_Pixel4
25,2020-05-15-US-MTV-1,Pixel4,1273608810662,37.416630,-122.081969,-10.05,2020-05-15-US-MTV-1_Pixel4
...,...,...,...,...,...,...,...
4661,2020-05-15-US-MTV-1,Pixel4XL,1273609925446,37.408569,-122.071247,-18.90,2020-05-15-US-MTV-1_Pixel4XL
4662,2020-05-15-US-MTV-1,Pixel4XL,1273609926446,37.408441,-122.071083,-15.48,2020-05-15-US-MTV-1_Pixel4XL
4663,2020-05-15-US-MTV-1,Pixel4XL,1273609927446,37.408318,-122.070924,-15.50,2020-05-15-US-MTV-1_Pixel4XL
4664,2020-05-15-US-MTV-1,Pixel4XL,1273609928446,37.408189,-122.070774,-14.87,2020-05-15-US-MTV-1_Pixel4XL


In [65]:
tmp[tmp['millisSinceGpsEpoch'].isin(base_test['millisSinceGpsEpoch'])]

,collectionName,phoneName,millisSinceGpsEpoch,constellationType,svid,signalType,receivedSvTimeInGpsNanos,xSatPosM,ySatPosM,zSatPosM,xSatVelMps,ySatVelMps,zSatVelMps,satClkBiasM,satClkDriftMps,rawPrM,rawPrUncM,isrbM,ionoDelayM,tropoDelayM
325,2020-05-15-US-MTV-1,Pixel4,1273608801654,1,2,GPS_L1,1273608800568117297,-1.355350e+07,-2.285558e+07,1.592091e+06,302.889,-47.613,3160.562,-136834.396,-0.002,2.175749e+07,3.598,0.000,4.784,3.446
326,2020-05-15-US-MTV-1,Pixel4,1273608801654,1,6,GPS_L1,1273608800571097922,-5.144400e+06,-2.379852e+07,1.060766e+07,823.829,1072.241,2823.713,-81575.055,-0.002,2.086392e+07,2.998,0.000,3.944,2.849
327,2020-05-15-US-MTV-1,Pixel4,1273608801654,6,25,GAL_E1,1273608800554999274,4.323120e+06,-2.881377e+07,5.220182e+06,23.951,-536.768,-2983.575,-153084.743,-0.000,2.569017e+07,2.398,-216.118,5.918,4.439
328,2020-05-15-US-MTV-1,Pixel4,1273608801654,1,17,GPS_L1,1273608800565023316,7.661722e+06,-1.391200e+07,2.170208e+07,2640.705,239.526,-798.715,79526.801,0.003,2.268504e+07,1.799,0.000,5.496,4.392
329,2020-05-15-US-MTV-1,Pixel4,1273608801654,3,22,GLO_G1,1273608800574904107,-3.862793e+06,-1.355721e+07,2.122717e+07,2429.314,1613.344,1483.924,-418.462,-0.001,1.972285e+07,4.797,1144.294,3.714,2.857
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57725,2020-05-15-US-MTV-1,Pixel4,1273612275632,3,13,GLO_G1,1273612275568180115,-1.219834e+07,-1.338065e+07,1.794654e+07,-705.894,-2360.645,-2237.702,-11610.677,0.000,1.927615e+07,4.197,1153.070,3.620,2.577
57726,2020-05-15-US-MTV-1,Pixel4,1273612275632,3,14,GLO_G1,1273612275557382661,-9.952700e+06,5.477318e+06,2.284841e+07,-718.551,-3041.501,414.853,15362.334,-0.000,2.251315e+07,4.797,1153.070,6.953,6.868
57727,2020-05-15-US-MTV-1,Pixel4,1273612275632,6,30,GAL_E1,1273612275555754963,-1.589697e+07,-7.064249e+06,2.393849e+07,22.881,-2404.899,-693.358,1170586.979,-0.009,2.300112e+07,5.096,-222.012,4.288,3.066
57728,2020-05-15-US-MTV-1,Pixel4,1273612275632,6,36,GAL_E1,1273612275552308959,6.525533e+05,-1.761160e+07,2.378304e+07,2044.321,1235.423,859.698,167805.129,-0.002,2.403420e+07,2.099,-222.012,4.311,3.081


In [54]:
tmp['millisSinceGpsEpoch'].describe()

count    5.773000e+04
mean     1.273611e+12
std      9.921158e+05
min      1.273609e+12
25%      1.273610e+12
50%      1.273611e+12
75%      1.273611e+12
max      1.273612e+12
Name: millisSinceGpsEpoch, dtype: float64

In [55]:
base_test['millisSinceGpsEpoch'].describe()

count    9.148600e+04
mean     1.289832e+12
std      1.296896e+10
min      1.273609e+12
25%      1.275847e+12
50%      1.299958e+12
75%      1.301953e+12
max      1.303773e+12
Name: millisSinceGpsEpoch, dtype: float64